In [1]:
transcript = """
Physician: Good morning, Ms. Jones. How are you feeling today?

Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.

Physician: I understand you were in a car accident last September. Can you walk me through what happened?

Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.

Physician: That sounds like a strong impact. Were you wearing your seatbelt?

Patient: Yes, I always do.

Physician: What did you feel immediately after the accident?

Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.

Physician: Did you seek medical attention at that time?

Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.

Physician: How did things progress after that?

Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.

Physician: That makes sense. Are you still experiencing pain now?

Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.

Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?

Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.

Physician: And how has this impacted your daily life? Work, hobbies, anything like that?

Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.

Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.

[Physical Examination Conducted]

Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.

Patient: That’s a relief!

Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.

Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?

Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.

Patient: Thank you, doctor. I appreciate it.

Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
"""

In [2]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
import re

/Users/gaganyadav/anaconda3/envs/deep_learning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Below Cell may take some time to run

In [4]:
def classify_utterance_by_rules(utterance: str) -> str:

    utterance_lower = utterance.lower()
    

    subjective_keywords = [
        'i feel', 'i have', 'my neck', 'my back', 'pain', 'discomfort', 'i realized',
        'i went to', 'i had to take', 'i was driving', 'hit me from behind', 'headache',
        'nauseous', 'worried', 'anxious', 'trouble sleeping', 'it hurts',"pain", "ache", "felt", "feeling", "sore", "discomfort", "tender", "stiffness", 
        "burning", "throbbing", "numb", "tingling", "swelling", "cramps", "dizzy", 
        "nausea", "vomit", "fatigue", "tired", "shortness of breath", "chills",
        "concern", "worried", "difficulty", "symptom", "complaint", "report", "mention"
    ]
    if any(re.search(r'\b' + keyword + r'\b', utterance_lower) for keyword in subjective_keywords):
        if "physician:" in utterance_lower and "?" in utterance_lower:
            return "Other"
    
        if "patient:" in utterance_lower:
            return "Subjective"
            
    objective_keywords = [
        'physical examination', 'looks good', 'range of movement', 'no tenderness',
        'signs of', 'muscles and spine', 'vitals show', 'exam reveals', 
        'mri shows', 'x-rays',   "exam", "examination", "physical", "inspection", "observed", "range of motion", 
        "palpated", "auscultation", "percussion", "inspection", "muscle strength", 
        "reflex", "tone", "vitals", "vital signs", "heart rate", "blood pressure", 
        "respiratory rate", "oxygen saturation", "temp", "imaging", "lab", "labs", 
        "x-ray", "mri", "ct scan", "ultrasound", "blood test", "found", "revealed", 
        "measured", "noted", "detected"
    ]
    if any(keyword in utterance_lower for keyword in objective_keywords):
      
        if "physician:" in utterance_lower:
            return "Objective"

    
    plan_keywords = [
        'i recommend', 'i’d expect', 'prognosis', 'full recovery', 'come back for a follow-up',
        'prescribe', 'referral', 'we will', 'let\'s go ahead and', 'i\'m going to',
        'schedule a', 'take care', 'don\'t hesitate to reach out',
        "recommend", "prescribe", "prescription", "medication", "medicine", "continue", 
        "start", "begin", "initiate", "refer", "referral", "therapy", "physical therapy", 
        "rehab", "exercise", "rest", "monitor", "reassess", "plan", "follow up", 
        "return", "check-up", "consult", "schedule", "come back", "if symptoms worsen", 
        "education", "advise", "suggest", "instruct", "counsel", "avoid", "stop", 
        "resume", "take", "use", "apply", "order", "administer"
    ]
    if any(keyword in utterance_lower for keyword in plan_keywords):
        if "physician:" in utterance_lower:
            return "Plan"

    return "Other"


class Summarizer:
    def __init__(self, model_name="facebook/bart-large-cnn"):
        """
        Initializes the BART summarizer.
        """
    
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.model = BartForConditionalGeneration.from_pretrained(model_name).to(self.device)
    

    def summarize(self, text: str, min_length=15, max_length=100) -> str:
       

        if not text or not text.strip():
            return "No information provided for this section."

        inputs = self.tokenizer([text], max_length=1024, return_tensors='pt', truncation=True).to(self.device)
        
        summary_ids = self.model.generate(
            inputs['input_ids'],
            num_beams=4,
            min_length=min_length,
            max_length=max_length,
            early_stopping=True
        )
        
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        return summary



def generate_soap_note(transcript: str, summarizer: Summarizer) -> dict:
  
    utterances = transcript.strip().split('\n')
    utterances = [u.strip() for u in utterances if u.strip()]
    
   
    classified_texts = {
        "Subjective": [],
        "Objective": [],
        "Plan": []
    }
    
    for u in utterances:
        category = classify_utterance_by_rules(u)
        if category in classified_texts:
            cleaned_utterance = re.sub(r'^(Physician|Patient):\s*', '', u)
            classified_texts[category].append(cleaned_utterance)

   
    subjective_text = " ".join(classified_texts["Subjective"])
    objective_text = " ".join(classified_texts["Objective"])
    plan_text = " ".join(classified_texts["Plan"])

    
    soap_note = {}
    soap_note["Subjective"] = summarizer.summarize(subjective_text, max_length=120)
    soap_note["Objective"] = summarizer.summarize(objective_text, max_length=80)
    soap_note["Plan"] = summarizer.summarize(plan_text, max_length=100)
    
    assessment_input = (
        f"Subjective: {soap_note['Subjective']} "
        f"Objective: {soap_note['Objective']} "
        f"Plan: {soap_note['Plan']}"
    )
    soap_note["Assessment"] = summarizer.summarize(assessment_input, min_length=10, max_length=60)
    
    return soap_note


summarizer_model = Summarizer()

final_note = generate_soap_note(transcript, summarizer_model)
    

print(f"Subjective:\n {final_note['Subjective']}\n")
print(f"Objective:\n {final_note['Objective']}\n")
print(f"Assessment:\n {final_note['Assessment']}\n")
print(f"Plan:\n {final_note['Plan']}\n")

Subjective:
 I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front. At first, I was just shocked. But then I realized I had hit my head on the steering wheel.

Objective:
 There are no signs of long-term damage or degeneration. Given your progress, I’d expect you to make a full recovery within six months of the accident.

Assessment:
 I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Another car hit me from behind, which pushed my car into the one in front. At first, I was just shocked. But then I realized I had hit my head on the steering

Plan:
 I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.

